In [2]:
import pandas as pd

df_wb_final = pd.read_csv('world_bank_data_processed.csv')
df_athlete_merged=pd.read_csv('athlete_data_processed.csv')
df_noc=pd.read_csv('noc_regions_processed.csv')

In [3]:
df_athlete_merged.columns

Index(['id_athlete', 'Name', 'Sex', 'Team', 'NOC', 'Games', 'Year', 'Season',
       'City', 'Sport', 'Event', 'Medal', 'region', 'medal_score'],
      dtype='object')

In [4]:
df_noc.columns

Index(['NOC', 'region'], dtype='object')

In [5]:
df_wb_final.columns

Index(['country', 'year', 'population_total', 'population_65_UP',
       'population_15_64', 'densite', 'pib_total', 'pib_par_habitant',
       'croissance_pib', 'participation_femmes', 'densite_urbaine',
       'population_urbaine', 'NOC'],
      dtype='object')

In [6]:
years = df_athlete_merged["Year"].unique()
df_wb_final = df_wb_final[df_wb_final["year"].isin(years)]
df_wb_final.shape

(3759, 13)

In [7]:
years

array([1992, 2012, 1920, 1900, 1988, 1994, 1932, 2002, 1952, 1980, 2000,
       1996, 1912, 1924, 2014, 1948, 1998, 2006, 2008, 2016, 2004, 1960,
       1964, 1984, 1968, 1972, 1936, 1956, 1928, 1976, 2010, 1906, 1904,
       1908, 1896], dtype=int64)

In [8]:
df_wb_final.isnull().sum()

country                    0
year                       0
population_total           0
population_65_UP           0
population_15_64           0
densite                  470
pib_total                473
pib_par_habitant         473
croissance_pib           605
participation_femmes    1666
densite_urbaine          882
population_urbaine        21
NOC                        0
dtype: int64

## Calcul des KPI

#### Médailles totales & Médaille-score pondéré

In [9]:
df_score_pondere = df_athlete_merged.groupby(["NOC", "Year"]).agg(
    total_medals = ("Medal", lambda x: (x != "No Medal").sum()),
    weighted_score = ("medal_score", "sum")
).reset_index()
df_score_pondere

,NOC,Year,total_medals,weighted_score
0,AFG,1936,0,0
1,AFG,1948,0,0
2,AFG,1956,0,0
3,AFG,1960,0,0
4,AFG,1964,0,0
...,...,...,...,...
3299,ZIM,2004,3,6
3300,ZIM,2008,4,9
3301,ZIM,2012,0,0
3302,ZIM,2014,0,0


In [10]:
df_wb_final = df_wb_final.merge(
    df_score_pondere,
    left_on=["NOC", "year"],
    right_on=["NOC", "Year"],
    how="inner"
)
df_wb_final.head()

,country,year,population_total,population_65_UP,population_15_64,densite,pib_total,pib_par_habitant,croissance_pib,participation_femmes,densite_urbaine,population_urbaine,NOC,Year,total_medals,weighted_score
0,Afghanistan,1960,9035043.0,255092.0,4990044.0,NaN,NaN,NaN,NaN,NaN,285352.0,8.401,AFG,1960,0,0
1,Afghanistan,1964,9814318.0,270658.0,5381453.0,15.047327,NaN,NaN,NaN,NaN,350382.0,9.586,AFG,1964,0,0
2,Afghanistan,1968,10756922.0,290464.0,5794810.0,16.492529,NaN,NaN,NaN,NaN,428363.0,10.918,AFG,1968,0,0
3,Afghanistan,1972,11853696.0,312828.0,6279251.0,18.174104,NaN,NaN,NaN,NaN,528508.0,12.410,AFG,1972,0,0
4,Afghanistan,1980,13169311.0,331463.0,6772533.0,20.191207,NaN,NaN,NaN,NaN,977824.0,15.995,AFG,1980,0,0


#### Médaille-score pondéré / million habitants

In [11]:
df_wb_final["weighted_score_per_million"] = df_wb_final.apply(
    lambda row: row["weighted_score"] / (row["population_total"] / 1000000)
    if row["population_total"] > 0 else None,
    axis=1
)
df_wb_final.head()

,country,year,population_total,population_65_UP,population_15_64,densite,pib_total,pib_par_habitant,croissance_pib,participation_femmes,densite_urbaine,population_urbaine,NOC,Year,total_medals,weighted_score,weighted_score_per_million
0,Afghanistan,1960,9035043.0,255092.0,4990044.0,NaN,NaN,NaN,NaN,NaN,285352.0,8.401,AFG,1960,0,0,0.0
1,Afghanistan,1964,9814318.0,270658.0,5381453.0,15.047327,NaN,NaN,NaN,NaN,350382.0,9.586,AFG,1964,0,0,0.0
2,Afghanistan,1968,10756922.0,290464.0,5794810.0,16.492529,NaN,NaN,NaN,NaN,428363.0,10.918,AFG,1968,0,0,0.0
3,Afghanistan,1972,11853696.0,312828.0,6279251.0,18.174104,NaN,NaN,NaN,NaN,528508.0,12.410,AFG,1972,0,0,0.0
4,Afghanistan,1980,13169311.0,331463.0,6772533.0,20.191207,NaN,NaN,NaN,NaN,977824.0,15.995,AFG,1980,0,0,0.0


#### Ratio médailles / urbanisation

In [12]:
df_wb_final["medals_per_urbanization"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["population_urbaine"]
    if row["population_urbaine"] not in [0, None] else None,
    axis=1
)
df_wb_final.head()

,country,year,population_total,population_65_UP,population_15_64,densite,pib_total,pib_par_habitant,croissance_pib,participation_femmes,densite_urbaine,population_urbaine,NOC,Year,total_medals,weighted_score,weighted_score_per_million,medals_per_urbanization
0,Afghanistan,1960,9035043.0,255092.0,4990044.0,NaN,NaN,NaN,NaN,NaN,285352.0,8.401,AFG,1960,0,0,0.0,0.0
1,Afghanistan,1964,9814318.0,270658.0,5381453.0,15.047327,NaN,NaN,NaN,NaN,350382.0,9.586,AFG,1964,0,0,0.0,0.0
2,Afghanistan,1968,10756922.0,290464.0,5794810.0,16.492529,NaN,NaN,NaN,NaN,428363.0,10.918,AFG,1968,0,0,0.0,0.0
3,Afghanistan,1972,11853696.0,312828.0,6279251.0,18.174104,NaN,NaN,NaN,NaN,528508.0,12.410,AFG,1972,0,0,0.0,0.0
4,Afghanistan,1980,13169311.0,331463.0,6772533.0,20.191207,NaN,NaN,NaN,NaN,977824.0,15.995,AFG,1980,0,0,0.0,0.0


#### Ratio sportivité / densité

In [13]:
df_wb_final["medals_per_density"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["densite"]
    if row["densite"] not in [0, None] else None,
    axis=1
)

#### Médailles / PIB

In [14]:
df_wb_final["medals_per_pib"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["pib_total"]
    if row["pib_total"] not in [0, None] else None,
    axis=1
)

#### Score pondéré / PIB_en_milliards

In [15]:
df_wb_final["weighted_score_per_pib_milliard"] = df_wb_final.apply(
    lambda row: row["weighted_score"] / (row["pib_total"] / 1e9)
    if row["pib_total"] not in [0, None] else None,
    axis=1
)

#### Score efficacité composite

In [16]:
df_wb_final["composite_efficiency_score"] = df_wb_final.apply(
    lambda row: (row["weighted_score"] / row["population_total"]) * row["pib_par_habitant"]
    if row["population_total"] not in [0, None] and row["pib_par_habitant"] not in [0, None]
    else None,
    axis=1
)

#### Médailles / PIB par habitant

In [17]:
df_wb_final["medals_per_gdp_per_capita"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["pib_par_habitant"]
    if row["pib_par_habitant"] not in [0, None] else None,
    axis=1
)

#### Ratio médailles / population active

In [18]:
df_wb_final["medals_per_working_population"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["population_15_64"]
    if row["population_15_64"] not in [0, None] else None,
    axis=1
)

#### Médailles / croissance PIB annuelle

In [19]:
df_wb_final["medals_per_gdp_growth"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["croissance_pib"]
    if row["croissance_pib"] not in [0, None, 0] else None,
    axis=1
)

#### Ratio médaille / densité urbaine

In [20]:
df_wb_final["medals_per_urban_density"] = df_wb_final.apply(
    lambda row: row["total_medals"] / row["densite_urbaine"]
    if row["densite_urbaine"] not in [0, None] else None,
    axis=1
)

#### Ratio participation / femmes

le nombre d’athlètes femmes par pays et par année

In [21]:
df_female_athletes = df_athlete_merged[df_athlete_merged["Sex"] == "F"] \
    .groupby(["NOC", "Year"]) \
    .agg(female_athletes=("id_athlete", "count")) \
    .reset_index()

In [22]:
df_female_athletes

,NOC,Year,female_athletes
0,AFG,2004,2
1,AFG,2008,1
2,AFG,2012,1
3,AFG,2016,1
4,AHO,1968,2
...,...,...,...
2457,ZIM,2000,8
2458,ZIM,2004,5
2459,ZIM,2008,8
2460,ZIM,2012,5


fusionner avec df_wb_final

In [23]:
df_wb_final = df_wb_final.merge(
    df_female_athletes,
    left_on=["NOC", "Year"],
    right_on=["NOC", "Year"],
    how="left"
).drop(columns=["Year"])


In [24]:
df_wb_final["female_athletes"] = df_wb_final["female_athletes"].fillna(0)

calcul de nbr de population femme

In [25]:
df_wb_final["population_femmes"] = (
    df_wb_final["population_15_64"] * (df_wb_final["participation_femmes"] / 100)
)

In [26]:
df_wb_final["ratio_female_participation"] = df_wb_final.apply(
    lambda row: row["female_athletes"] / row["population_femmes"]
    if row["population_femmes"] not in [0, None] else None,
    axis=1
)

#### Taux participation aux JO par habitant (Athlètes ÷ population 15–64 ans)

Nombre total d’athlètes par pays + année

In [28]:
df_athletes = df_athlete_merged.groupby(["NOC", "Year"]) \
    .agg(total_athletes=("id_athlete", "count")) \
    .reset_index()

fusionner avec df_wb_final

In [29]:
df_wb_final = df_wb_final.merge(
    df_athletes,
    left_on=["NOC", "year"],
    right_on=["NOC", "Year"],
    how="left"
).drop(columns=["Year"])

In [30]:
df_wb_final["total_athletes"] = df_wb_final["total_athletes"].fillna(0)

In [31]:
df_wb_final["participation_rate_active_pop"] = df_wb_final.apply(
    lambda row: row["total_athletes"] / row["population_15_64"]
    if row["population_15_64"] not in [0, None] else None,
    axis=1
)

#### création de dataframe de KPI

In [49]:
df_wb_final.columns

Index(['country', 'year', 'population_total', 'population_65_UP',
       'population_15_64', 'densite', 'pib_total', 'pib_par_habitant',
       'croissance_pib', 'participation_femmes', 'densite_urbaine',
       'population_urbaine', 'NOC', 'total_medals', 'weighted_score',
       'weighted_score_per_million', 'medals_per_urbanization',
       'medals_per_density', 'medals_per_pib',
       'weighted_score_per_pib_milliard', 'composite_efficiency_score',
       'medals_per_gdp_per_capita', 'medals_per_working_population',
       'medals_per_gdp_growth', 'medals_per_urban_density', 'female_athletes',
       'population_femmes', 'ratio_female_participation', 'total_athletes',
       'participation_rate_active_pop'],
      dtype='object')

In [50]:
cols_to_drop = [
    'population_total', 'population_65_UP', 'population_15_64', 'densite',
    'pib_total', 'pib_par_habitant', 'croissance_pib', 'participation_femmes',
    'densite_urbaine', 'population_urbaine'
]

df_wb_kpi = df_wb_final.drop(columns=cols_to_drop)

In [51]:
df_wb_kpi.to_csv('kpi_world_bank_data.csv', index=False)

In [52]:
df_wb_kpi.columns

Index(['country', 'year', 'NOC', 'total_medals', 'weighted_score',
       'weighted_score_per_million', 'medals_per_urbanization',
       'medals_per_density', 'medals_per_pib',
       'weighted_score_per_pib_milliard', 'composite_efficiency_score',
       'medals_per_gdp_per_capita', 'medals_per_working_population',
       'medals_per_gdp_growth', 'medals_per_urban_density', 'female_athletes',
       'population_femmes', 'ratio_female_participation', 'total_athletes',
       'participation_rate_active_pop'],
      dtype='object')

In [53]:
df_wb_kpi.isnull().sum()

country                              0
year                                 0
NOC                                  0
total_medals                         0
weighted_score                       0
weighted_score_per_million           0
medals_per_urbanization              1
medals_per_density                 131
medals_per_pib                     112
weighted_score_per_pib_milliard    112
composite_efficiency_score         112
medals_per_gdp_per_capita          112
medals_per_working_population        0
medals_per_gdp_growth              177
medals_per_urban_density           446
female_athletes                      0
population_femmes                  877
ratio_female_participation         877
total_athletes                       0
participation_rate_active_pop        0
dtype: int64

In [54]:
df_wb_kpi.shape

(2243, 20)

In [55]:
df_wb_kpi[df_wb_kpi["country"]=="Germany"]

,country,year,NOC,total_medals,weighted_score,weighted_score_per_million,medals_per_urbanization,medals_per_density,medals_per_pib,weighted_score_per_pib_milliard,composite_efficiency_score,medals_per_gdp_per_capita,medals_per_working_population,medals_per_gdp_growth,medals_per_urban_density,female_athletes,population_femmes,ratio_female_participation,total_athletes,participation_rate_active_pop
779,Germany,1960,GER,98,211,2.897759,1.372857,NaN,1.158121e-09,2.493505,0.003368,0.084328,2.001593e-06,NaN,0.000030,137.0,NaN,NaN,535,0.000011
780,Germany,1964,GER,126,224,2.974043,1.752653,0.584059,1.021646e-09,1.816259,0.004870,0.076949,2.569967e-06,18.977419,0.000039,156.0,NaN,NaN,622,0.000013
781,Germany,1968,FRG,61,118,1.526632,0.845496,0.275530,3.897084e-10,0.753862,0.003092,0.030122,1.243949e-06,10.760706,0.000019,135.0,NaN,NaN,569,0.000012
782,Germany,1968,GDR,58,110,1.423132,0.803914,0.261980,3.705424e-10,0.702753,0.002882,0.028641,1.182772e-06,10.231491,0.000018,104.0,NaN,NaN,431,0.000009
783,Germany,1972,FRG,112,229,2.910211,1.547196,0.496929,3.722168e-10,0.761050,0.011128,0.029289,2.254745e-06,26.044444,0.000035,182.0,NaN,NaN,709,0.000014
784,Germany,1972,GDR,171,308,3.914170,2.362237,0.758704,5.682952e-10,1.023596,0.014968,0.044718,3.442513e-06,39.764286,0.000054,151.0,NaN,NaN,523,0.000011
785,Germany,1976,FRG,110,188,2.399889,1.514797,0.490245,2.108658e-10,0.360389,0.015981,0.016519,2.193416e-06,22.225552,0.000035,142.0,NaN,NaN,557,0.000011
786,Germany,1976,GDR,227,512,6.535868,3.125990,1.011687,4.351504e-10,0.981485,0.043523,0.034088,4.526413e-06,45.865457,0.000073,216.0,NaN,NaN,504,0.000010
787,Germany,1980,FRG,8,10,0.127733,0.109824,0.035676,8.387744e-12,0.010485,0.001556,0.000657,1.549587e-07,5.678476,0.000003,31.0,NaN,NaN,118,0.000002
788,Germany,1980,GDR,303,664,8.481442,4.159574,1.351237,3.176858e-10,0.696183,0.103328,0.024871,5.869059e-06,215.072286,0.000099,219.0,NaN,NaN,582,0.000011


In [62]:
df_wb_kpi[df_wb_kpi["country"]=="Germany"]["total_medals"].sum()

3083